In [1]:
#%matplotlib inline
import pandas as pd
import json
import geopandas as gpd
import plotly.express as px

In [2]:
import plotly.io as pio
print(pio.renderers.default)

plotly_mimetype+notebook


In [2]:
#read in csv file
df_nta = pd.read_csv(r'C:\Users\steve\GitHub\rp-covid-migration\data\time_away_ntas_2020.csv')

#melt for animation format
list_date = df_nta.columns.tolist()
#list_date = list_date[61:122] # march 1st to may 31st
#list_date = list_date[61:63] # march 1st to may 31st
df = pd.melt(df_nta, id_vars=['NTA'],value_vars = list_date)
df = df.rename(columns={'NTA':'id'}) #rename so the join ids are the same
df['variable'] = df.variable.str.replace('-','/')
df['val_pct'] = df['value']*100 # convert it into percentage
df.tail(20)

Exception: Data must be 1-dimensional

In [24]:
df['val_pct'].describe()

count    17296.000000
mean         3.279158
std          2.883630
min          0.000000
25%          1.853653
50%          2.584720
75%          3.595376
max         27.328431
Name: val_pct, dtype: float64

In [4]:
#read in geojson file
map_ntas = json.load(open(r'C:\Users\steve\GitHub\rp-covid-migration\data\shapefiles\NTA map.geojson','r'))

#check properties
map_ntas['features'][0]['properties']

{'ntacode': 'BK43',
 'shape_area': '35799637.8103',
 'ntaname': 'Midwood',
 'shape_leng': '27996.5912736',
 'boroname': 'Brooklyn',
 'borocode': '3',
 'countyfips': '047'}

In [5]:
# add feature id to geojson file
for feature in map_ntas['features']:
    feature['id'] = feature['properties']['ntacode']

In [6]:
# check to make sure id is working
map_ntas['features'][0]['id']

'BK43'

In [13]:
nbhood = gpd.read_file(r'C:\Users\steve\GitHub\rp-covid-migration\data\shapefiles\nynta.shp')
nbhood.drop(columns = ['BoroCode','CountyFIPS','Shape_Leng','Shape_Area','geometry'])
df = pd.merge(df, nbhood, left_on = 'id', right_on = 'NTACode', how = 'inner')
#df = df.drop(columns = ['NTACode'])
df = df.rename({'NTAName':'neighborhood', 'variable':'Date', 'val_pct':'percent_away'}, axis = 1)
df['neighborhood'] = df['neighborhood'].str.replace('-',' ') # replace - value with space
df.head()
df.to_csv(r'C:\Users\steve\GitHub\rp-covid-migration\data\pct_time_away.csv')

In [16]:
#converting from df to GeoJSON
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [17]:
gdf.to_file(r"C:\Users\steve\GitHub\rp-covid-migration\data\shapefiles\time_away_nta.geojson", driver='GeoJSON')

In [47]:
fig = px.choropleth_mapbox(df, geojson = map_ntas,
                   locations ='id',
                   color = 'val_pct',
                   hover_name = 'neighborhood',
                   animation_frame ='Date',
                   color_continuous_scale = "Reds",
                   opacity = 0.8,
                   labels={'val_pct':'% Away >22hrs'},
                   range_color = [0,30])
fig.add_annotation(text = 'Percentage Resident Devices<br>Away from Home > 22 hours',
                  align = 'right', x = 0.08, y = 0.95,
                  showarrow = False,
                  bordercolor = None,
                  bgcolor = None,
                  opacity = 0.95,
                  font = dict(family = "Droid Serif", color = 'black', size = 28 ))
#fig.add_annotation(text = 'Residents Away from Home',
 #                 align = 'right', x = 0.08, y = 0.90,
  #                showarrow = False,
   #               bordercolor = None,
    #              bgcolor = None,
     #             opacity = 0.95,
      #            font = dict(family = "Droid Serif", color = 'black', size = 28 ))
fig.add_annotation(text = 'From March 1st to May 31st',
                  align = 'right', x = 0.1, y = 0.86,
                  showarrow = False,
                  bordercolor = None,
                  bgcolor = None,
                  opacity = 0.95,
                  font = dict(family = "Droid Serif", color = 'black', size = 24 ))
fig.update_layout(mapbox_zoom = 9.42,
                 mapbox_center = {'lat':40.7, 'lon':-73.95},
                 mapbox_style="carto-positron",
                 margin = {'r':1,'t':1,'l':1,'b':1},
                 legend = dict(yanchor = "top", y = 0.8, xanchor = "left", x = 0.10))
# title left side of map
# alter legend
#fig.show()
fig.write_html('pct_time_away.html', auto_open = True, auto_play = False, include_plotlyjs = 'cdn')